Part 3: Data Analytics

0. Load both the csv file from Part 1 pr.data.0.Current and the json file from Part 2 as dataframes (Spark, Pyspark, Pandas, Koalas, etc).

In [3]:
# Importing necessary libraries
import pandas as pd
import requests

# Data sources from S3
BCL_TIME_SERIES_DATA = "https://rearc-asessment.s3.us-east-2.amazonaws.com/pr.data.0.Current"
POPULATION_DATA = "https://rearc-asessment.s3.us-east-2.amazonaws.com/population.json"

# Loading the Data in Dataframes using Pandas(I could also have used Pyspark for quicker computing but as the data is small using pandas is realibale beacuse of cost efficiency )
bcl = pd.read_csv(BCL_TIME_SERIES_DATA, sep="\t")
r = requests.get(POPULATION_DATA).json()
population = pd.json_normalize(r, record_path="data")

In [4]:
#We will perform data cleaning in this section
#Standardizing column names and trimming whispaces in the column names
bcl = bcl.rename(columns=lambda c: c.strip().lower())
population = population.rename(columns=lambda c: c.strip().lower().replace(' ', '_'))
#Trim any whitespaces
for df in (bcl, population):
    str_cols = df.select_dtypes(include="object").columns
    for c in str_cols:
        df[c] = df[c].str.strip()

#Drop any BCL rows that have missing data
bcl = bcl.dropna(subset=['series_id', 'year', 'period', 'value'])
#we don’t need footnote codes, so we can drop them
bcl = bcl.drop(columns=['footnote_codes'])
#Droping the duplictes for the bcl time series data
bcl = bcl.drop_duplicates(subset = ['series_id', 'year', 'period'],keep = 'last').reset_index(drop = True)


#For population, drop rows which has missing population or year
population = population.dropna(subset=['year', 'population'])
#Converting datatypes for the required columns (This step is currently not necssary but in future it may happen that datatypes are not correct therefore as precaution I am doing this transformation)
population["year"]       = population["year"].astype(int)
population["population"] = population["population"].astype(int)

1. Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.

In [5]:
# First we need to filter population data across the years [2013, 2018] inclusive
population_filtered = population[(population["year"] >= 2013) & (population["year"] <= 2018)]
# Calucalte the mean and standard deviation from the filtered population
population_filtered["population"].mean(), population_filtered["population"].std()

(322069808.0, 4158441.040908095)

2. Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year.

In [6]:
#Sum values by (series_id, year)
yearly = (bcl.groupby(["series_id", "year"], as_index=False)["value"].sum().rename(columns={"value": "total_value"}))

#For each series_id, pick the row where total_value is maximum
best_year = (yearly.loc[yearly.groupby("series_id")["total_value"].idxmax()].reset_index(drop=True))

best_year = best_year.rename(columns={"total_value": "value"})

3. Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset).

In [7]:
# Filter the data according to required attributes and it's value
subset = (bcl[(bcl["series_id"] == "PRS30006032") & (bcl["period"] == "Q01")].copy())

#Merge the data with respect to year
result = pd.merge(subset,population,left_on  = "year",right_on = "year",how = "left")

# Select the required columns and create report
report = result[["series_id", "year", "period", "value", "population"]]

In [8]:
best_year

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2024,860.838
279,PRS88003201,2022,37.200
280,PRS88003202,2022,28.700


In [9]:
report

,series_id,year,period,value,population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.3,NaN
7,PRS30006032,2002,Q01,-6.6,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.0,NaN
